In [ ]:
import pandas as pd
import xarray as xr
import glob
import os

from windwhisper import *

In [ ]:
# we can preload the wind speed data, otherwise, the tool will do it every time
filepath_wind_speed = "/Users/romain/GitHub/windwhisper/dev/fixtures/era5_mean_2013-2022_month_by_hour.nc"
filepath_correction = "/Users/romain/GitHub/windwhisper/dev/fixtures/ratio_gwa2_era5.nc"

def wind_speed_data():
    wind_speed = xr.open_dataset(filepath_wind_speed).to_array().mean(dim="month")
    correction = xr.open_dataset(filepath_correction).to_array()
    correction = correction.sel(variable='ratio_gwa2_era5_mean_WS').interp(latitude=wind_speed.latitude, longitude=wind_speed.longitude, method="linear")
    return wind_speed #* correction
    
data = wind_speed_data()

elevation_data = xr.open_dataset("../fixtures/Copernicus_DSM_90m_COG.nc")

In [ ]:
dirs = [
    "4x4_highhigh",
    "4x4_lowlow",
    "4x4_medmed"
]

In [ ]:
dfs = pd.DataFrame()

for directory in dirs:
    full_path = os.path.join(directory)
    csv_files = glob.glob(os.path.join(full_path, "*.csv"))
    
    for file in csv_files:
        df = pd.read_csv(file)
        df['source_directory'] = directory
        df['source_file'] = os.path.basename(file)
        dfs = pd.concat([dfs, df], ignore_index=True)

In [ ]:
for scenario in dfs["source_directory"].unique():
    for polygon in dfs.loc[dfs["source_directory"]==scenario, "source_file"].unique():
        print(f"Processing {scenario} - {polygon}")
        df = dfs.loc[
            (dfs["source_directory"]==scenario)
            &(dfs["source_file"]==polygon)
        ]
        df.loc[:, "rated_power_MW"] *= 1000

        wind_turbines = {
            f'Turbine {i}': {
                'diameter': row['rotor_diameter'],
                'hub height': row['hub_heights'],
                'position': (row['lat'], row['lon']),
                'power': row['rated_power_MW']
            }
            for i, row in df.iterrows()
        }

        wt = WindTurbines(
            wind_turbines=wind_turbines,
            wind_speed_data=data,
        )
        
        noise_prop = NoisePropagation(
            wind_turbines=wt.wind_turbines,
            humidity=70,
            temperature=20,
            elevation_data=elevation_data,
        )
        
        noise_analysis = NoiseAnalysis(
            noise_propagation=noise_prop,
            wind_turbines=wt.wind_turbines,
        )

        noise_analysis.merged_map.rio.write_crs("EPSG:4326", inplace=True)
        noise_analysis.merged_map.to_netcdf(f"{scenario}_{polygon.replace('.csv', '')}.nc")

In [ ]:
import xarray as xr
arr = xr.open_dataset("4x4_highhigh_best_polygon_polygon_1.nc")

In [ ]:
print(arr.rio.crs)

In [ ]:
import numpy as np

# Example assuming you have the dataset loaded as ds
lon_res = np.diff(np.unique(arr.lon.values)).mean()
lat_res = np.diff(np.unique(arr.lat.values)).mean()

print(f"Longitude resolution: {lon_res:.5f} degrees")
print(f"Latitude resolution: {lat_res:.5f} degrees")


In [ ]:
arr["net"].plot.contour()